In [1]:
from pyscf import gto, scf, cc

a = 2 # 2aB
nH = 20
atoms = ""
for i in range(nH):
    atoms += f"H {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="6-31g", unit='bohr', spin=0, verbose=4)
mol.build()

mf = scf.RHF(mol).density_fit()
mf.kernel()

mo = mf.stability()[0]
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)
mo = mf.stability()[0]
dm = mf.make_rdm1(mo,mf.mo_occ)
mf.kernel(dm0=dm)


nfrozen = 0
mycc = cc.CCSD(mf,frozen=nfrozen)
mycc.kernel()[0]

System: uname_result(system='Linux', node='sharmagroup-rn', release='6.14.0-37-generic', version='#37~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov 20 10:25:38 UTC 2', machine='x86_64')  Threads 16
Python 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
numpy 2.3.1  scipy 1.16.2  h5py 3.14.0
Date: Fri Feb  6 19:28:13 2026
PySCF version 2.11.0
PySCF path  /home/sharmagroup/sharmagroup/pyscf
GIT HEAD (branch master) 3d1768f5e33b144b606c3d2c81c12ee54d794501

[ENV] PYSCF_EXT_PATH /home/sharmagroup/sharmagroup/pyscf-forge
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 20
[INPUT] num. electrons = 20
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = bohr
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 B

np.float64(-0.3476953019617713)

In [2]:
from pyscf import lo
import numpy as np
nocc = np.count_nonzero(mf.mo_occ)
orbocc = mf.mo_coeff[:,nfrozen:nocc]
lo_coeff  = lo.PipekMezey(mol, orbocc).kernel()

frag_lolist = [[i] for i in range(nocc)]



******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
pop_method = meta_lowdin
Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 4.0867237175129  delta_f= 4.08672  |g|= 0.993069  2 KF 6 Hx
macro= 2  f(x)= 4.0904216412529  delta_f= 0.00369792  |g|= 0.000790692  2 KF 5 Hx
macro= 3  f(x)= 4.0904216412544  delta_f= 1.42997e-12  |g|= 2.3969e-05  1 KF 1 Hx
macro X = 3  f(x)= 4.0904216412544  |g|= 2.3969e-05  6 intor 5 KF 12 Hx


In [3]:
from collections.abc import Iterable
from pyscf.lno import lnoccsd
# from ad_afqmc.lno_afqmc import ulno_afqmc

thresh = 1e-5
run_frg_list = [0]
chol_cut = 1e-5

mlno = lnoccsd.LNOCCSD(mf, lo_coeff, frag_lolist, frozen=nfrozen).set(verbose=4)
mlno.lno_thresh = [thresh*10,thresh]
lno_thresh = mlno.lno_thresh
lno_type = ['1h','1h']
lno_thresh = [1e-5, 1e-6] if lno_thresh is None else lno_thresh
print(lno_thresh)
lno_pct_occ = None
lno_norb = None
lo_proj_thresh = 1e-10
lo_proj_thresh_active = 0.1
eris = None

if run_frg_list is not None:
    frag_lolist = [frag_lolist[i] for i in run_frg_list]

nfrag = len(frag_lolist)
print(f'number of fragment {nfrag}')
print(f'fragment {frag_lolist}')

if lno_pct_occ is None:
    lno_pct_occ = [None, None]
if lno_norb is None:
    lno_norb = [[None,None]] * nfrag
mf = mlno._scf

if eris is None: eris = mlno.ao2mo()

# Loop over fragment
for ifrag, loidx in enumerate(frag_lolist):
    if len(loidx) == 2 and isinstance(loidx[0], Iterable): # Unrestricted
        orbloc = [lo_coeff[0][:,loidx[0]], lo_coeff[1][:,loidx[1]]]
        lno_param = [
            [
                {
                    'thresh': (
                        lno_thresh[i][s] if isinstance(lno_thresh[i], Iterable)
                        else lno_thresh[i]
                    ),
                    'pct_occ': (
                        lno_pct_occ[i][s] if isinstance(lno_pct_occ[i], Iterable)
                        else lno_pct_occ[i]
                    ),
                    'norb': (
                        lno_norb[ifrag][i][s] if isinstance(lno_norb[ifrag][i], Iterable)
                        else lno_norb[ifrag][i]
                    ),
                } for i in [0, 1]
            ] for s in range(2)
        ]

    else:
        orbloc = lo_coeff[:,loidx]
        lno_param = [{'thresh': lno_thresh[i], 'pct_occ': lno_pct_occ[i],
                        'norb': lno_norb[ifrag][i]} for i in [0,1]]

    lno_coeff, lno_frozen, uocc_loc, frag_msg = mlno.make_las(eris, orbloc, lno_type, lno_param)

[0.0001, 1e-05]
number of fragment 1
fragment [[0]]
LO occ proj: 1 active | 0 standby | 9 orthogonal


LO vir proj: 0 active | 0 standby | 30 orthogonal


In [40]:
lno_thresh

[0.0001, 1e-05]

In [4]:
mol = mf.mol
nocc = np.count_nonzero(mf.mo_occ)
actfrag = np.array([i for i in range(mol.nao) if i not in lno_frozen])
frzocc = np.array([i for i in range(nocc) if i in lno_frozen])
actocc = np.array([i for i in range(nocc) if i in actfrag])
actvir = np.array([i for i in range(nocc,mol.nao) if i in actfrag])
nfrzocc = len(frzocc)
nactocc = len(actocc)
nactvir = len(actvir)
nactorb = len(actfrag)

ncas = nactorb
ncore = nfrzocc
nelec = nactocc*2

print(nelec,ncas,ncore)

12 24 4


In [5]:
mo_occ = mlno.mo_occ
lno_frozen, maskact = lnoccsd.get_maskact(lno_frozen, mo_occ.size)

mf.max_memory = 0
mcc = lnoccsd.CCSD(mf, mo_coeff=lno_coeff, frozen=lno_frozen).set(verbose=4)
mcc._s1e = mlno._s1e
mcc._h1e = mlno._h1e
mcc._vhf = mlno._vhf
if mlno.kwargs_imp is not None:
    mcc = mcc.set(**mlno.kwargs_imp)

mcc.verbose = 4
from pyscf.lno import lnoccsd
(elcorr_pt2, elcorr_cc, elcorr_cc_t), frag_msg \
    = lnoccsd.impurity_solve(mcc, lno_coeff, uocc_loc, mo_occ, maskact, eris, log=None,
                             ccsd_t=False, verbose_imp=5)

Init t2, MP2 energy = -10.8073526019486  E_corr(MP2) -0.136611615535209

******** <class 'pyscf.lno.lnoccsd.MODIFIED_CCSD'> ********
CC2 = 0
CCSD nocc = 6, nmo = 24
frozen orbitals [ 0  1  2  3 28 29 30 31 32 33 34 35 36 37 38 39]
max_cycle = 50
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-05
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 0 MB (current use 236 MB)
Init E_corr(MODIFIED_CCSD) = -0.136611615535209
cycle = 1  E_corr(MODIFIED_CCSD) = -0.163245578596128  dE = -0.0266339631  norm(t1,t2) = 0.0739705
cycle = 2  E_corr(MODIFIED_CCSD) = -0.173893521966103  dE = -0.0106479434  norm(t1,t2) = 0.0442132
cycle = 3  E_corr(MODIFIED_CCSD) = -0.181134070248931  dE = -0.00724054828  norm(t1,t2) = 0.0259414
cycle = 4  E_corr(MODIFIED_CCSD) = -0.184640307970143  dE = -0.00350623772  norm(t1,t2) = 0.014261
cycle = 5  E_corr(MODIFIED_CCSD) = -0.184175429418279  dE = 0.000464878552  norm(t1,t2) = 0.00482109
cycle = 6  E_corr(MODIFIED_CCSD) = -0.184117798555936

In [6]:
imp_eris = mcc.ao2mo()

In [7]:
imp_eris.fock.shape

(24, 24)

In [8]:
from ad_afqmc.lno_afqmc import lno_afqmc
h1e, enuc = lno_afqmc.h1e_ras(mf, lno_coeff, ncas, ncore)

# Building JK matrix


In [ ]:
from pyscf import lib

def h1e_ras(mf, mo_coeff, ncas, ncore):
    '''
    effective one-electron integral for restricted active space
    ncas = nact_electron/2
    ncore = ncore_electrons/2
    '''
    # note casci undo DF

    mo_core = mo_coeff[:,:ncore]
    mo_cas = mo_coeff[:,ncore:ncore+ncas]

    hcore = mf.get_hcore() # h1 in ao
    energy_core = mf.energy_nuc() # h0
    if mo_core.size == 0:
        corevhf = 0.
    else:
        # core_dm = np.dot(mo_core, mo_core.T)
        core_dm = mo_core @ mo_core.T
        corevhf = lno_afqmc.get_veff(mf, core_dm)
        energy_core += 2 * lib.einsum('ij,ji', core_dm, hcore)
        energy_core += lib.einsum('ij,ji', core_dm, corevhf)
    h1eff = mo_cas.T @ (hcore+corevhf) @ mo_cas
    return h1eff, energy_core

In [9]:
print(h1e)

[[-2.83964704e+00 -3.32930988e-03  1.25253397e-01  3.18246447e-03
   4.24645458e-02  4.35877429e-03 -1.26689269e-03  6.51828368e-04
   4.59606742e-03 -1.27737050e-03 -7.11345812e-03  3.71312316e-03
  -5.56875229e-03  2.55874577e-03 -1.80957415e-01  1.01005939e-02
  -4.64037656e-02  2.57850411e-02  4.38758057e-03 -4.97093308e-03
   1.18405704e-03  2.43744924e-04  1.26346318e-03 -1.30732468e-03]
 [-3.32930988e-03 -2.66025457e+00  5.77842694e-03 -2.15510073e-01
  -9.87810616e-03 -4.82753541e-02 -7.25812075e-03 -1.90837531e-03
  -8.71189212e-04 -1.82602552e-02 -7.77693741e-03 -6.88326136e-03
  -8.68184538e-03 -3.40184682e-03  2.66010120e-02  6.51808234e-02
  -1.36832947e-01 -8.62627168e-02 -3.62096679e-02  4.76182241e-03
   7.11208826e-03 -1.60196263e-03 -1.13827635e-03 -3.34048157e-04]
 [ 1.25253397e-01  5.77842694e-03 -2.47475350e+00 -1.69746325e-02
   3.06060338e-01  1.18065076e-02  2.49560801e-02  9.73672683e-03
   2.61182825e-02 -1.14047642e-02  6.32064133e-03 -6.92126795e-04
   7.466

In [28]:
imp_fock = imp_eris.fock

In [17]:
mcc.mo_coeff-lno_coeff

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(40, 40))

In [20]:
mcc.mo_occ.shape

(40,)

In [10]:
dm = mf.make_rdm1(mcc.mo_coeff, mcc.mo_occ)
vhf = imp_eris.get_vhf(mf,dm)

In [11]:
mo_cas = lno_coeff[:,ncore:ncore+ncas]
vhf = mo_cas.T @ vhf @ mo_cas

In [ ]:
vhf 

array([[ 5.55008599e+00,  3.20317024e-03, -7.08018287e-02,
         3.07105384e-03,  4.61556102e-02,  2.73640610e-03,
        -2.63793905e-02, -8.15921904e-05, -2.85469776e-02,
         1.22825621e-04,  1.60841513e-02, -1.67117261e-03,
         2.72981890e-03, -6.33889191e-03,  3.77963710e-01,
        -1.25690334e-02,  9.46616751e-02, -4.12361117e-02,
        -1.51012900e-02,  1.24317507e-02,  4.54201361e-03,
         3.81954472e-03, -3.66119987e-03, -8.81854065e-04],
       [ 3.20317024e-03,  5.50453449e+00, -3.03219499e-03,
         1.43516741e-01,  3.26117396e-03,  7.47536814e-02,
         4.33870953e-03,  7.17433069e-02,  5.34175450e-03,
         5.42887124e-02,  4.33085331e-03,  6.52487687e-04,
         2.70681555e-02,  1.99541874e-02, -6.12211790e-02,
        -1.42994635e-01,  3.00388126e-01,  1.76557101e-01,
         6.27298604e-02, -3.10315894e-02, -1.65940892e-02,
        -1.27969555e-02, -6.23510416e-04,  4.66663311e-03],
       [-7.08018287e-02, -3.03219499e-03,  5.46363734e

In [80]:
print('# Calculating Effective Active Space One-electron Integrals')
mo_coeff = lno_coeff
frozen = lno_frozen
mol = mf.mol
nocc = np.count_nonzero(mf.mo_occ)
actfrag = np.array([i for i in range(mol.nao) if i not in frozen])
frzocc = np.array([i for i in range(nocc) if i in frozen])
actocc = np.array([i for i in range(nocc) if i in actfrag])
actvir = np.array([i for i in range(nocc,mol.nao) if i in actfrag])
nfrzocc = len(frzocc)
nactocc = len(actocc)
nactvir = len(actvir)
nactorb = len(actfrag)

ncas = nactorb
ncore = nfrzocc
nelec = nactocc*2
# h1e, enuc = h1e_ras(mf, mo_coeff, ncas, ncore)
mo_act = mo_coeff[:,actfrag]

# Calculating Effective Active Space One-electron Integrals
